## *SECTION 1 / SPRINT 1 / NOTE 2*

---

# Feature Engineering

## 학습 목표 

- Feature Engineering 의 목적을 이해 할 수 있다.
- pandas를 통해 문자열(string)을 다룰 수 있다.
- 데이터프레임에 `.apply()`를 사용하여 행을 수정하거나 새로 작업 할 수 있다.

---

## Feature Engineering이란



### 개요

Feature Engineering 은 도메인 지식과 창의성을 바탕으로, 데이터셋에 존재하는 Feature들을 재조합하여 새로운 Feature를 만드는 것입니다. 

<img src='https://i.imgur.com/0IW7xm8.png' width = 500>

가령 위 그림처럼 2개의 feature를 가진 데이터가 있다면, 

해당 feature를 조합하여 (`+`)

새로운 feature를 만들어 낸 다음, 이를 분석에 사용 할 수 있을 것 입니다.

여기서 조합은, 방금 말한것처럼 간단한 연산 일 수도 있고

<img src='https://i.imgur.com/AHnak4A.png' width = 500>

이미 실생활에서 사용 하고 있는 것처럼 조금 더 복잡한 연산 일 수도 있습니다.


통계 분석 혹은 머신러닝, 더 나아가 딥러닝까지 대부분의 분석은 데이터에 있는 패턴을 인식하고, 해당 패턴들을 바탕으로 예측을 하기 때문에, 

더 좋은 퍼포먼스를 위하여 더 새롭고, 더 의미있는 패턴을 제공하는 것이 궁극적인 Feature engineering의 목적입니다.



### DataFrame

그 전에 우리가 점차 익숙해져가는 pandas의 dataframe에 대한 설명을 간단하게 하겠습니다.

지금은 그냥 **테이블 형태의 데이터다** 정도로만 이해하셔도 충분합니다.

<img src='https://i.imgur.com/w694Hye.png' width = 500>

일반적으로 하나의 행에는 하나의 데이터 혹은 관측치.

하나의 열에는 하나의 feature를 기반으로 저장하기를 권장합니다.

이를 tidy 형태 라고 부르기도 하며, 이는 이후에 조금 더 배우겠지만 다른 라이브러리들과의 호환성을 위해서 라 생각하면 좋습니다.

### Dataset

오늘 feature engineering을 연습하기 위해 사용할 데이터는 kt&g 라는 기업의 일부 재무정보 데이터이며 형태는 다음과 같습니다.

<img src='https://i.imgur.com/fHop57h.png' width = 500>

<https://finance.naver.com/item/coinfo.nhn?code=033780&target=finsum_more>



In [1]:
import pandas as pd

df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/kt%26g/kt%26g.csv')

df

분기     매출액   영업이익 영업이익(발표기준) 세전계속사업이익  당기순이익 당기순이익(지배)  당기순이익(비지배)  \
0   19/6  12,578  3,991      3,991    4,452  3,245     3,228          17   
1   19/9  13,222  3,825      3,825    4,503  3,221     3,208          13   
2  19/12  11,982  2,523      2,523    1,794  1,183     1,198         -15   
3   20/3  11,784  3,150      3,150    4,156  2,939     2,930           9   
4   20/6  13,188  3,947      3,947    4,020  2,935     2,931           4   

      자산총계    부채총계  ... 투자활동현금흐름 재무활동현금흐름  영업이익률   순이익률 ROE(%) ROA(%)   부채비율  \
0  108,464  25,940  ...   -1,878   -4,919  31.73  25.80  12.35   9.37  31.43   
1  106,314  20,691  ...      -83     -329  28.93  24.36  12.67   9.95  24.17   
2  107,121  20,062  ...     -501     -129  21.06   9.87  12.39   9.96  23.04   
3  108,594  24,862  ...   -1,392      -86  26.73  24.94  13.05   9.79  29.69   
4  110,282  23,386  ...     -214   -5,501  29.93  22.25  12.20   9.40  26.91   

    자본유보율  EPS(원)  PER(배)  
0  803.22   2,351   13.77  
1  836.86   2,337   13.87  
2  850.82     873   12.42  
3  823.16   2,134    9.71  
4  837.40   2,135   10.46  

[5 rows x 25 columns]

In [ ]:
df.dtypes

분기             object
매출액            object
영업이익           object
영업이익(발표기준)     object
세전계속사업이익       object
당기순이익          object
당기순이익(지배)      object
당기순이익(비지배)      int64
자산총계           object
부채총계           object
자본총계           object
자본총계(지배)       object
자본총계(비지배)     float64
자본금            object
영업활동현금흐름       object
투자활동현금흐름       object
재무활동현금흐름       object
영업이익률         float64
순이익률          float64
ROE(%)        float64
ROA(%)        float64
부채비율          float64
자본유보율         float64
EPS(원)         object
PER(배)        float64
dtype: object

데이터는 전부 숫자형 데이터지만 

그 형태가 `object`, `float64`, `int64`등으로 다르게 표현되어 있습니다.



In [ ]:
df['자본총계(비지배)']

0      NaN
1      NaN
2      NaN
3    562.0
4    566.0
Name: 자본총계(비지배), dtype: float64

일단 자본총계 (비지배) 부분을 집중해서 보면, `float64`로 표현되어 있습니다.

값들이 "진짜" 실수형 값인가요?

위에 쓰여진 NaN은, Not a Number의 줄임말로 pandas에서 결측치를 표현하는 방법입니다.

NaN은 프로그래밍상 float라는 type을 갖습니다.

이 때문에 562, 566은 int -> float로 어쩔 수 없이 type cast가 된 것입니다.


### 🔥 Na, Null, NaN, 0, Undefined 의 차이를 알아보세요.

### Feature Engineering

|순이익률|ROE(%)|ROA(%)|
|:-:|:-:|:-:|
|25.80|12.35|9.37|
|24.36|12.67|9.95|
|9.87|12.39|9.96|
|24.94|13.05|9.79|
|22.25|12.20|9.40|

위 데이터를 기반으로 새로운 투자지표 

"J-value" = `ROE` + `ROA`

를 만들어 보겠습니다.


In [ ]:
df['j-value'] = df['ROE(%)'] + df['ROA(%)']
df

분기     매출액   영업이익 영업이익(발표기준)  ...   자본유보율 EPS(원) PER(배)  j-value
0   19/6  12,578  3,991      3,991  ...  803.22  2,351  13.77    21.72
1   19/9  13,222  3,825      3,825  ...  836.86  2,337  13.87    22.62
2  19/12  11,982  2,523      2,523  ...  850.82    873  12.42    22.35
3   20/3  11,784  3,150      3,150  ...  823.16  2,134   9.71    22.84
4   20/6  13,188  3,947      3,947  ...  837.40  2,135  10.46    21.60

[5 rows x 26 columns]

In [ ]:
df['자산'] = df['부채총계'] + df['자본총계']
df

분기     매출액   영업이익 영업이익(발표기준)  ... EPS(원) PER(배) j-value            자산
0   19/6  12,578  3,991      3,991  ...  2,351  13.77   21.72  25,94082,524
1   19/9  13,222  3,825      3,825  ...  2,337  13.87   22.62  20,69185,623
2  19/12  11,982  2,523      2,523  ...    873  12.42   22.35  20,06287,059
3   20/3  11,784  3,150      3,150  ...  2,134   9.71   22.84  24,86283,732
4   20/6  13,188  3,947      3,947  ...  2,135  10.46   21.60  23,38686,896

[5 rows x 27 columns]

## String

### 개요

`25,970` + `82,524` = `108,464` 가 되어야 합니다.

그렇지만 우리의 결과는 `25,97082,524` 였습니다.

<img src='https://i.imgur.com/80AOnF5.png' width = 500>

이번에는 문자열(string)을 다루는 방법에 대해서 배워보겠습니다.

### 프로세스 소개

<img src='https://i.imgur.com/WYQhrs8.png' width = 300>

앞서 발생한 문제를 해결 하기 위해서, 다음과 같은 단계를 거쳐야합니다.

1. 문자를 숫자로 바꾸기 위해 **숫자가 아닌 부분을 제거**

2. 문자를 숫자로 **형변환**

일반적으로 머신러닝 모델링에서는 문자열로 이루어진 값은 사용하지 않습니다. (매우 복잡한 문제입니다.)

### string replace

<https://www.w3schools.com/python/ref_string_replace.asp>

`replace`의 사용법은 다음과 같습니다.

**string variable**.replace(“삭제할 글자“, ‘’) 의 형태로 사용 ( 공백으로 대치 )

`s.replace(',', '')`

### 🔥 <https://www.w3schools.com/python/python_strings.asp>


In [ ]:
testString = '25,970'

In [ ]:
testString.replace(',','')

'25970'

In [ ]:
testString

'25,970'

### Type casting

<img src='https://i.imgur.com/Y15ZJoL.png' width = 400>

In [ ]:
testString = testString.replace(',','')

In [ ]:
int(testString)

25970

### as Function

<https://www.w3schools.com/python/python_functions.asp>

In [ ]:
# 입력된 문자열에 대해서 같은 작업을 하는 함수 작성
def toInt(string):
    return int(string.replace(',',''))

In [ ]:
# 예시 데이터를 바탕으로 함수를 테스트
toInt('25,970')

25970

In [ ]:
type(toInt('25,970'))

int

## Apply

### 개요

<img src='https://i.imgur.com/n4vvIDr.png' width = 500>

데이터의 모든 문자열에 대해서 일일히 `toInt`함수를 반복 할 수는 없음.

대신 `column` 단위로 할 수 있지 않을까??

<https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html>

### apply 사용법

1. apply 안에 들어갈 함수를 선언
2. column에 apply 적용.

In [ ]:
df['자산2'] = df['자산'].apply(toInt)

df

분기     매출액   영업이익 영업이익(발표기준)  ... PER(배) j-value            자산         자산2
0   19/6  12,578  3,991      3,991  ...  13.77   21.72  25,94082,524  2594082524
1   19/9  13,222  3,825      3,825  ...  13.87   22.62  20,69185,623  2069185623
2  19/12  11,982  2,523      2,523  ...  12.42   22.35  20,06287,059  2006287059
3   20/3  11,784  3,150      3,150  ...   9.71   22.84  24,86283,732  2486283732
4   20/6  13,188  3,947      3,947  ...  10.46   21.60  23,38686,896  2338686896

[5 rows x 28 columns]

In [ ]:
df['부채총계'] = df['부채총계'].apply(toInt)
df['자본총계'] = df['자본총계'].apply(toInt)

df['자산'] = df['부채총계'] + df['자본총계']

df

분기     매출액   영업이익 영업이익(발표기준)  ... PER(배) j-value      자산         자산2
0   19/6  12,578  3,991      3,991  ...  13.77   21.72  108464  2594082524
1   19/9  13,222  3,825      3,825  ...  13.87   22.62  106314  2069185623
2  19/12  11,982  2,523      2,523  ...  12.42   22.35  107121  2006287059
3   20/3  11,784  3,150      3,150  ...   9.71   22.84  108594  2486283732
4   20/6  13,188  3,947      3,947  ...  10.46   21.60  110282  2338686896

[5 rows x 28 columns]